In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver6 import DatasetGeneratorVer6
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver7 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [3]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3"
outdir = Path("results/distilbert-base-uncased-ver7")
fold = 3

In [4]:
with open("config/distilbert-base-uncased-ver7.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased-ver7'),
 'debug': False,
 'seed': 20210425,
 'ver': 7,
 'train': {'SAM': False,
  'folds': [3],
  'checkpoints': ['results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05_3.pth'],
  'loss': {'name': 'focal',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 12,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ver6'),
  'batch_size': 35,
  'num_workers': 8,
  'oversampling': True,
  'oversampling_scale': 10}}

In [6]:
trainer = MyTrainer(C, fold, outdir / f"{postfix}.pth")

Load pretrained results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3.pth


In [7]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [8]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [9]:
model.pre_classifier.register_forward_hook(hook)

In [10]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [ ]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

 76%|████████████████████████████████████████▉             | 319130/421079 [15:50<05:03, 336.34it/s]

In [ ]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [ ]:
deck["tlevel"].shape, deck["fclevel"].shape

In [ ]:
deck["feat"].shape

In [ ]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [14]:
deck = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck1.pth")

In [15]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [16]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-4.8830,  4.1110, -5.7687, -4.4678, -7.3731, -4.0645, -6.7607]])


In [17]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[24.6197, 32.4965, 36.9424, 40.4677]]),
 tensor([[144, 177, 112, 195]]),
 tensor([[0, 0, 0, 0]]))

In [18]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-3.2189, -2.4237, -7.7600,  1.8829, -8.6021, -5.1994, -7.5476]])


In [19]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[38.2679, 40.8461, 46.9950, 47.6562]]),
 tensor([[536, 236, 272, 377]]),
 tensor([[0, 0, 0, 0]]))

In [20]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-0.8619, -2.2384, -1.8922, -2.0378, -3.1984, -2.5976, -2.0431]])


In [21]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[29.5083, 43.1404, 52.9118, 53.9867]]),
 tensor([[144, 103,  82, 757]]),
 tensor([[0, 0, 0, 0]]))

# 덱만들기 (Submission)

In [22]:
ds_test = trainer.dsgen.test()

In [23]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [1:42:37<00:00, 177.99it/s]


In [ ]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [38]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 1, 768]), torch.Size([1095951]))

In [39]:
deck["feat"] = deck["feat"][:, 0, :]
# deck["fclevel"] = deck["fclevel"][:, 0]

In [40]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 768]), torch.Size([1095951]))

In [41]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")